## Get texts from link got from 01

Edit "id" ,"devide" and "drivers_n"

Data format : (time)(devide)(text content)
Default:(time)@@@(text content)

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import chromedriver_binary
from concurrent import futures

In [2]:
id = "Mitsubishi"

link_file = "Link/PantipLink_%s.txt"%id
text_file = "TextData/PantipTextData_%s.txt"%id

devide = "@@@"

This will use several browsers in the same time for speed.
Thus, this will take CPU resource.
Watch your Task Manager and adjust "drivers_n" where CPU won't be occupied 100%

In [3]:
drivers_n = 7   #Adjust!

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")

#Open Several drivers
drivers = []
for _ in range(drivers_n):
    drivers.append(webdriver.Chrome(options=chrome_options))

drivers_used = [False for _ in range(drivers_n)]

In [4]:
def scrape(url):
    global scraped, scrape_max, drivers_n, drivers_used, drivers, f

    #find open driver
    for index in range(drivers_n):
        if drivers_used[index] == False:
            using_index = index
            drivers_used[index] = True
            driver = drivers[index]
            break

    #Open Page
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
 
    posts = soup.find_all(class_="display-post-status-leftside")
 
    #Scrape texts
    for post in posts:

        el = post.find("div", class_="display-post-story")
    
        #get post time
        if post.find("abbr") == None:
            continue
        time = post.find("abbr").get("data-utime")
        
        #erace edit history
        for history in el.find_all("div", class_ = "edit-history"):
            history.decompose()

        #make it clean
        texts = el.text.replace("\t", " ").replace("\n", " ").split(" ")
        text = "".join(texts)
       
        #save
        try:
            f.write(time + devide + text + "\n")
        except Exception as e:
            print(e)
            print(text)
    
    drivers_used[using_index] = False

    #show progress
    scraped += 1
    if (scraped / scrape_max) % 0.05  < 0.005:
        print((scraped/scrape_max)*100, "%")


In [5]:
links_f = open(link_file, "r")

links = links_f.read().split("\n")[:-1]

links_f.close()

f = open(text_file, "w", encoding="utf-8")

#scrape texts in the same time
scraped = 0
scrape_max = len(links)
future_list = []
with futures.ThreadPoolExecutor(max_workers=drivers_n) as executor:
    for link in links:
        future = executor.submit(scrape, url=link)
        future_list.append(future)
    _ = futures.as_completed(fs=future_list)
 
f.close()

0.00018772292096865028
0.050122019898629626
0.1000563168762906
0.1501783367749202
0.2001126337525812
0.25004693073024215
0.3001689506288718
0.3501032476065328
0.4000375445841937
0.45015956448282335
0.5000938614604843
0.5500281584381453
0.600150178336775
0.6500844753144359
0.7000187722920969
0.7501407921907265
0.8000750891683874
0.8500093861460485
0.8501971090670171
0.900131406044678
0.9500657030223391
